In [1]:
%matplotlib inline
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

#### Knuckle Map

In [2]:
keypointsMapping = ['Nose', 'Neck', 'R-Sho', 'R-Elb', 'R-Wr', 'L-Sho', 'L-Elb', 'L-Wr', 'R-Hip', 'R-Knee', 'R-Ank', 'L-Hip', 'L-Knee', 'L-Ank', 'R-Eye', 'L-Eye', 'R-Ear', 'L-Ear']
POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7], [1,8], [8,9], [9,10], [1,11], [11,12], [12,13], [1,0], [0,14], [14,16], [0,15], [15,17]]
nPoints = 18
threshold = 0.1

#### Network

In [3]:
pose_detection = cv2.dnn.readNet("human-pose-estimation-0001.bin",
                           "human-pose-estimation-0001.xml")

person_detection = cv2.dnn.readNet("person-detection-retail-0013.bin",
                                  "person-detection-retail-0013.xml")


In [4]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = 25
cap.set(cv2.CAP_PROP_FRAME_WIDTH,912)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,512)


# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
writer = cv2.VideoWriter('MultiPersonDetection.mp4', cv2.VideoWriter_fourcc(*'DIVX'), fps, (width, height))

while cv2.waitKey(1) < 0:
    ret, frame = cap.read()
    if not ret:
        break
    
    try:
        start_time = time.time()
        person_bolb = cv2.dnn.blobFromImage(frame,size = (300,300))
        person_detection.setInput(person_bolb)
        person_output = person_detection.forward()


        for detection in person_output.reshape(-1, 7):
            points = []
            confidence = float(detection[2])
            xmin = int(detection[3] * frame.shape[1])
            ymin = int(detection[4] * frame.shape[0])
            xmax = int(detection[5] * frame.shape[1])
            ymax = int(detection[6] * frame.shape[0])

            if confidence > 0.7:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color=(0, 255, 0),thickness = 3)
                detection_box = frame[ymin:ymax,xmin:xmax]

                pose_blob = cv2.dnn.blobFromImage(detection_box,size =(256,456))
                pose_detection.setInput(pose_blob)
                pose_output = pose_detection.forward(['Mconv7_stage2_L1', 'Mconv7_stage2_L2'])

                frameWidth = detection_box.shape[1]
                frameHeight = detection_box.shape[0]
                H = pose_output[1].shape[2]
                W = pose_output[1].shape[3]          

                for i in range(18):
                    probMap = pose_output[1][0,i,:,:]

                    minVal, maxVal, minLoc, point = cv2.minMaxLoc(probMap)

                    #print("MaxVal")
                    #print(maxVal)

                    # Scale the point to fit on the original image
                    x = (frameWidth * point[0]) / W
                    y = (frameHeight * point[1]) / H

                    if maxVal > threshold:
                        points.append((int(x),int(y)))

                    else:
                        points.append(None)


                for pair in POSE_PAIRS:
                    partA = pair[0]
                    partB = pair[1]
                    #print(partA)
                    #print(partB)


                    if points[partA] and points[partB]:
        #                 print(points[partA],points[partB])
                        cv2.line(detection_box,points[partA],points[partB],(0,255,0),3)   
            
        end_time = time.time()
        fps = np.round(1/(end_time-start_time))
        
        cv2.putText(frame,text="Current FPS: {}".format(fps),
        org=(0,30),  # X. Y
        fontFace=font,fontScale= 0.5,color=(0,0,255),thickness=2,lineType=cv2.LINE_AA)        
            
        
        cv2.imshow('OpenVINO person detection', frame)
            
    except:
        cv2.imshow('OpenVINO person detection', frame)
        pass
    
    writer.write(frame)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break 
            
# When everything done, realease the capture and destroy the windows
cap.release()
writer.release()
cv2.destroyAllWindows()